In [1]:
import os
orig_working_directory=os.getcwd()
orig_working_directory

'C:\\Users\\clemi\\Desktop\\Clementine-GWU-HW\\ETL_Project'

In [2]:
os.chdir(os.path.join('..'))
curr_working_directory=os.getcwd()
curr_working_directory

'C:\\Users\\clemi\\Desktop\\Clementine-GWU-HW'

## Import Dependencies

In [3]:
import json
from sqlalchemy import create_engine
import pandas as pd
import pymysql
pymysql.install_as_MySQLdb()

## Import configuration variables

In [4]:
from salesforce_config import sf_username, sf_password, sf_security_token
from salesforce_config import remote_db_endpoint, remote_db_port
from salesforce_config import remote_db_name, remote_db_user, remote_db_pwd

## Set up simple_salesforce

In [5]:
from simple_salesforce import Salesforce
sf = Salesforce(username=sf_username, password=sf_password, security_token=sf_security_token)

## Connect to your MySQL database

In [6]:
engine = create_engine(f"mysql://{remote_db_user}:{remote_db_pwd}@{remote_db_endpoint}:{remote_db_port}/{remote_db_name}")
conn = engine.connect()

## Query Data

In [7]:
participant_data = pd.read_sql("SELECT * FROM classparticipant", conn)
participant_data.head(30)

,ID_ClassParticipant,ID_Student,ID_Class,StartDate,EndDate
0,1,33,1,2020-03-16,None
1,2,34,1,2020-03-16,None
2,3,35,1,2020-03-16,None
3,4,62,1,2020-03-16,None
4,5,36,1,2020-03-16,None
5,6,37,1,2020-03-16,None
6,7,38,1,2020-03-16,None
7,8,39,1,2020-03-16,None
8,9,40,1,2020-03-16,None
9,10,41,1,2020-03-16,None


## Rename Columns

In [8]:
participant_data=participant_data.rename(columns={"ID_ClassParticipant":"Name","ID_Student":"ID_Student__c",
                                        "ID_Class":"ID_Class__c","StartDate":"StartDate__c","EndDate":"EndDate__c"})
#convert date to string otherwise we will get JSON nonserilizable error
participant_data["StartDate__c"]=participant_data["StartDate__c"].astype(str)
participant_data.head()

,ID_ClassParticipant__c,Name,ID_Class__c,StartDate__c,EndDate__c
0,1,33,1,2020-03-16,None
1,2,34,1,2020-03-16,None
2,3,35,1,2020-03-16,None
3,4,62,1,2020-03-16,None
4,5,36,1,2020-03-16,None


## Convert DataFrame to a list of Dictionaries

In [9]:
participant_load = participant_data.to_dict('records')
participant_load

[{'ID_ClassParticipant__c': 1,
  'Name': 33,
  'ID_Class__c': 1,
  'StartDate__c': '2020-03-16',
  'EndDate__c': None},
 {'ID_ClassParticipant__c': 2,
  'Name': 34,
  'ID_Class__c': 1,
  'StartDate__c': '2020-03-16',
  'EndDate__c': None},
 {'ID_ClassParticipant__c': 3,
  'Name': 35,
  'ID_Class__c': 1,
  'StartDate__c': '2020-03-16',
  'EndDate__c': None},
 {'ID_ClassParticipant__c': 4,
  'Name': 62,
  'ID_Class__c': 1,
  'StartDate__c': '2020-03-16',
  'EndDate__c': None},
 {'ID_ClassParticipant__c': 5,
  'Name': 36,
  'ID_Class__c': 1,
  'StartDate__c': '2020-03-16',
  'EndDate__c': None},
 {'ID_ClassParticipant__c': 6,
  'Name': 37,
  'ID_Class__c': 1,
  'StartDate__c': '2020-03-16',
  'EndDate__c': None},
 {'ID_ClassParticipant__c': 7,
  'Name': 38,
  'ID_Class__c': 1,
  'StartDate__c': '2020-03-16',
  'EndDate__c': None},
 {'ID_ClassParticipant__c': 8,
  'Name': 39,
  'ID_Class__c': 1,
  'StartDate__c': '2020-03-16',
  'EndDate__c': None},
 {'ID_ClassParticipant__c': 9,
  'Name':

## Inserting rows in bulk

In [10]:
try:
    sf.bulk.ClassParticipant__c.insert(participant_load)
except Exception as e:
    print(e)